In [1]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();


In [2]:
await history.addMessage(new HumanMessage("你好"));
await history.addMessage(new AIMessage("你好啊！"));

In [3]:
const messages = await history.getMessages();

console.log(messages);

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好啊！", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好啊！",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [1]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [5]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";

const chatModel = new ChatAlibabaTongyi({
model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
temperature: 1,
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", `你是一个乐于助人的助手，尽你所能地回答所有问题。
  你很健谈，能从上下文中提供大量具体细节。
  如果你不知道问题的答案，那就如实地说你不知道。`],
  new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);

In [7]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("你好，我是叮当猫"));

const res1 = await chain.invoke({
    history_message: await history.getMessages(),
});

console.log(res1);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "你好，叮当猫！很高兴见到你。如果你有任何问题或者需要帮助，请随时告诉我。",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "你好，叮当猫！很高兴见到你。如果你有任何问题或者需要帮助，请随时告诉我。",
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 64, completionTokens: 20, totalTokens: 84 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [8]:
await history.addMessage(res1)
await history.addMessage(new HumanMessage("我是谁"));

In [9]:
const res2 = await chain.invoke({
    history_message: await history.getMessages(),
});
console.log(res2);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "你自称是叮当猫，这通常指的是卡通角色《蓝皮鼠和大脸猫》中的角色，或者在一些衍生作品中，也可能指代哆啦A梦（Doraemon）系列里的叮当猫（Ding Dong Bell）。如果你在玩某个角色扮演游戏或"... 20 more characters,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "你自称是叮当猫，这通常指的是卡通角色《蓝皮鼠和大脸猫》中的角色，或者在一些衍生作品中，也可能指代哆啦A梦（Doraemon）系列里的叮当猫（Ding Dong Bell）。如果你在玩某个角色扮演游戏或"... 20 more characters,
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 96, completionTokens: 73, totalTokens: 169 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [10]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const chatModel = new ChatAlibabaTongyi({
model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
temperature: 1,
});

const prompt = ChatPromptTemplate.fromMessages([
["system", "你是一个乐于助人的助手，尽你所能地回答所有问题。"],
new MessagesPlaceholder("history_message"),
["human","{input}"]
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
runnable: chain,
getMessageHistory: (_sessionId) => history,
inputMessagesKey: "input",
historyMessagesKey: "history_message",
});

In [11]:
const res1 = await chainWithHistory.invoke({
    input: "你好，我是叮当猫",
},{
    configurable: { sessionId: "none" }
});
console.log(res1);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "你好，叮当猫！很高兴认识你。如果你有任何问题或者需要帮助，请随时告诉我，我会尽力提供支持。",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "你好，叮当猫！很高兴认识你。如果你有任何问题或者需要帮助，请随时告诉我，我会尽力提供支持。",
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 35, completionTokens: 25, totalTokens: 60 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [12]:
const res2 = await chainWithHistory.invoke({
  input: "我是谁",
},{
  configurable: { sessionId: "none" }
});
console.log(res2);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "你自称是叮当猫，这通常是指卡通角色《蓝猫淘气三千问》中的主角之一，或者指《哆啦A梦》（Doraemon）中的重要角色，哆啦A梦的神奇小口袋里的道具“叮当”（铜锣）。如果你在玩角色扮演游戏或者有其他特定"... 36 more characters,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "你自称是叮当猫，这通常是指卡通角色《蓝猫淘气三千问》中的主角之一，或者指《哆啦A梦》（Doraemon）中的重要角色，哆啦A梦的神奇小口袋里的道具“叮当”（铜锣）。如果你在玩角色扮演游戏或者有其他特定"... 36 more characters,
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 72, completionTokens: 90, totalTokens: 162 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}


In [3]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableSequence, RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { getBufferString } from "@langchain/core/messages";

In [16]:
const summaryModel = new ChatAlibabaTongyi({
    model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
    temperature: 1,
});

const summaryPrompt = ChatPromptTemplate.fromTemplate(`
    逐步总结所提供的对话内容，在之前的总结基础上添加新的总结
    
    当前摘要:
    {summary}
    
    新的对话方式:
    {new_lines}
    
    新摘要:
    `); 

const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
]);

In [17]:
const newSummary = await summaryChain.invoke({
    summary: "",
    new_lines: "我是叮当猫"
});

In [18]:
await summaryChain.invoke({
  summary: newSummary,
  new_lines: "我会飞"
});

'在先前的对话总结中，我们主要探讨了整体情况和可能的话题范围。随着新角色"我是叮当猫"的加入，对话可能转向与超自然能力、冒险、童话世界或者科幻元素相关的内容，特别是如果"我会飞"的能力成为讨论的核心。然'... 40 more characters

In [19]:
const chatModel = new ChatAlibabaTongyi({
    model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
    temperature: 1,
});

const chatPrompt = ChatPromptTemplate.fromMessages([
    ["system", `你是一个乐于助人的助手，尽你所能地回答所有问题。

    以下是聊天记录摘要：
    {history_summary}
    `],
    ["human","{input}"]
]);

let summary = ""
const history = new ChatMessageHistory();

In [20]:
const chatChain = RunnableSequence.from([
    {
        input: new RunnablePassthrough({
             func: (input) => history.addUserMessage(input)
        })
    },
    RunnablePassthrough.assign({
        history_summary: () => summary
    }),
    chatPrompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (input) => {
            history.addAIChatMessage(input);
            const messages = await history.getMessages();
            const new_lines = getBufferString(messages);
            const newSummary = await summaryChain.invoke({
                summary,
                new_lines
            });
            history.clear();
            summary = newSummary;   
        }
    })
]);

In [21]:
const res1 = await chatChain.invoke("我现在饿了");
console.log(res1);

如果你饿了，可以考虑吃点东西。你可以选择制作简单的三明治、热汤或者是一碗方便面，如果你在家；外出的话，可以去附近的餐馆、便利店或者点外卖。别忘了喝点水哦。


In [22]:
const res2 = await chatChain.invoke("我今天想吃方便面");
console.log(res2);

当然可以！如果你想要吃方便面，这里有一些建议：

1. **选择口味**：确保挑选你喜欢的口味，比如经典的红烧牛肉、豚骨、泡椒凤爪或者蔬菜味等。

2. **准备材料**：根据包装上的指示，准备好所需的配料，如调料包、蔬菜、鸡蛋（如果需要）等。

3. **烧水煮面**：烧一锅开水，按照方便面包装上的时间煮面，通常大约3-5分钟。

4. **添加配料**：煮好面后，把调料包、蔬菜或其他喜欢的配料加入，搅拌均匀。

5. **享受美食**：你的快速方便面就完成了，记得配一碗热水或茶，补充一下水分。

6. **健康小贴士**：虽然方便面快捷，但为了均衡营养，偶尔可以搭配一些蔬菜、水果或低脂酸奶来提升饱腹感和营养。

享用你的方便面时，别忘了适时休息，补充能量哦！


In [24]:
const res3 = await history.getMessages();
console.log(res3);

[]


In [5]:
import { BufferMemory } from "langchain/memory";
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";

const chatModel = new ChatAlibabaTongyi({
    model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
    temperature: 1,
    verbose:true
});

const memory = new BufferMemory();

const TEMPLATE = `
你是一个乐于助人的 ai 助手，尽你所能回答所有问题。

这是跟人类沟通的聊天历史:
{history}

据此回答人类的问题:
{input}
`
const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

In [6]:
let tempInput = ""

  const chain =  RunnableSequence.from([
    {
      input: new RunnablePassthrough(),
      memoryObject: async (input) => {
        const history = await memory.loadMemoryVariables({
          input,
        });
        tempInput = input;
        return history;
      }
    },

    RunnablePassthrough.assign({
      history: (input) => input.memoryObject.history
    }),

    prompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
      func: async (output) => {
        await memory.saveContext({
          input: tempInput,
        }, {
          output,
        })
      }
    }),
  ]);

In [3]:
import { JSONChatHistory } from "./JSONChatHistory/index.ts"
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const history = new JSONChatHistory({
    dir: "chat_data",
    sessionId: "test"
})


await history.addMessages([
  new HumanMessage("你好，我是叮当猫"),
  new AIMessage("你好"),
]);

const messages = await history.getMessages();
console.log(messages)

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你好，我是叮当猫",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好，我是叮当猫",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [4]:
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";


const chatModel = new ChatAlibabaTongyi({
    model: "qwen-turbo", // Available models: qwen-turbo, qwen-plus, qwen-max
    temperature: 1,
});
const memory = new BufferMemory({
    chatHistory: history
});
const chain = new ConversationChain({ llm: chatModel, memory: memory });
const res1 = await chain.call({ input: "我是谁？" });
console.log(res1);

{
  response: "你是在说中文的“叮当猫”，这通常是中国动画《哆啦A梦》中的主角，一个来自22世纪的机器猫，以你的名字“Ding Dang”进行交流。"
}
